In [2]:
import numpy as np
from lookup_table import CaseNum2EdgeOffset, getCaseNum
import trimesh
import os
import time

In [11]:
def marching_cube(thres,cells):
    # vertices use dictionary to avoid duplicate axes
    vertex_array = {}
    face_array = []
    t1 = time.time()
    # -------------------TODO------------------ 
    # compute vertices and faces
    # vertices: [N, 3]
    # faces: [M, 3], e.g. np.array([[0,1,2]]) means a triangle composed of vertices[0], vertices[1] and vertices[2]
    # for-loop is allowed to reduce difficulty
    vertex_num = 0
    #print(cells.shape)
    for x in range(cells.shape[0] - 1):
        for y in range(cells.shape[1] - 1):
            for z in range(cells.shape[2] - 1):
                case_values = getCaseNum(x, y, z, thres, cells)
                calulated_case_values = {}
                vertex_idx = []
                for case_value in case_values:
                    if case_value == -1:
                        continue
                    if case_value in calulated_case_values:
                        vertex_idx.append(calulated_case_values[case_value])
                        if len(vertex_idx) >= 3:
                            face_array.append([vertex_idx[-1], vertex_idx[-2], vertex_idx[-3]])
                        continue

                    v_1 = [x + CaseNum2EdgeOffset[case_value][0], y + CaseNum2EdgeOffset[case_value][1], \
                           z + CaseNum2EdgeOffset[case_value][2]]
                    v_2 = [x + CaseNum2EdgeOffset[case_value][3], y + CaseNum2EdgeOffset[case_value][4], \
                           z + CaseNum2EdgeOffset[case_value][5]]
                    cell_1 = cells[v_1[0]][v_1[1]][v_1[2]]
                    cell_2 = cells[v_2[0]][v_2[1]][v_2[2]]
                    t = (thres - cell_2) / (cell_1 - cell_2)
                    v = [v_1[0] * t + v_2[0] * (1 - t), v_1[1] * t + v_2[1] * (1 - t), \
                         v_1[2] * t + v_2[2] * (1 - t)]
                    
                    vertex_array['{}'.format(vertex_num)] = v
                    calulated_case_values[case_value] = vertex_num
                    vertex_idx.append(vertex_num)
                    vertex_num += 1
                    if len(vertex_idx) >= 3:
                        face_array.append([vertex_idx[-1], vertex_idx[-2], vertex_idx[-3]])
                        #print(face_array[-1])
    # -------------------TODO------------------ 
    t2 = time.time()
    print("\nTime taken by algorithm\n"+'-'*40+"\n{} s".format(t2-t1))
    vertex_array = list(vertex_array.values())
    return np.array(vertex_array), np.array(face_array)

In [12]:
# reconstruct these two animals
shape_name_lst = ['spot', 'bob']
for shape_name in shape_name_lst:
    data = np.load(os.path.join('data', shape_name + '_cell.npy'))
    verts, faces = marching_cube(0, data)
    mesh = trimesh.Trimesh(vertices=verts, faces=faces)
    mesh_txt = trimesh.exchange.obj.export_obj(mesh)
    with open(os.path.join('../results', shape_name + '.obj'),"w") as fp:
        fp.write(mesh_txt)


Time taken by algorithm
----------------------------------------
3.6915693283081055 s

Time taken by algorithm
----------------------------------------
3.7124135494232178 s
